# Process Description: Obtaining Fundamentus Share Data

## 1. Web Scraping with Selenium
- Utilize Selenium, a web automation tool, to navigate to the Fundamentus website.
- Identify and locate the necessary HTML elements containing the shares data.
- Use Selenium to interact with the webpage, extract the required information such as stock prices, financial metrics, and other relevant data.

## 2. Downloading Files
- Identify and locate any downloadable files on the Fundamentus website, such as the general balance report of each company.
- Use Selenium to simulate the process of clicking download buttons or links.
- Implement logic to wait for file downloads to complete before proceeding.

## 3. Extracting from ZIP
- If the downloaded files are in ZIP format, use Python's zipfile module to programmatically extract the contents.
- Identify the target files within the ZIP archive that contain the shares data.
- Extract the relevant files to a specified directory for further processing.

## 4. Data Processing
- Once the shares data is obtained, processed, and extracted, proceed with any necessary data cleaning, transformation, or analysis.
- Utilize Python libraries such as Pandas for efficient data manipulation.

## 5. Data Storage or Analysis
- Depending on the project requirements, store the processed shares data in a database, XLS file, or other data storage solutions.
- Conduct further analysis, visualization, or reporting as needed.

### So, let's code it!

## Import the needed libraries

In [239]:
# Selenium for web automation
from selenium import webdriver  # pip install selenium or pip install --upgrade selenium
# Webdriver Manager for managing browser drivers
from webdriver_manager.chrome import ChromeDriverManager  # pip install webdriver-manager
# Selenium service for Chrome browser
from selenium.webdriver.chrome.service import Service
# Selenium common locator strategies (By)
from selenium.webdriver.common.by import By
# Time module for handling time-related operations
import time
# NumPy for numerical operations
import numpy as np
# PyAutoGUI for automating mouse and keyboard interactions
import pyautogui
# Selenium Keys for keyboard interactions
from selenium.webdriver.common.keys import Keys
# OS module for interacting with the operating system
import os
# shutil for high-level file operations
import shutil
# sys for accessing Python interpreter variables
import sys
# zipfile for working with ZIP archives
import zipfile
# Custom module for store the paths
import config
# import pandas fro data mining
import pandas as pd
# library to get the Yahoo cotations
import yfinance as yf 
# import math do deal with nan
import math
# deal with excel files
import xlrd
import xlwt
from xlutils.copy import copy

## Steps 1, 2 and 3 (Web Scraping with Selenium, Downloading Files and Extracting from ZIP)

### Automate the process of finding and downloading the files. Moving to desired folder and extracting to XLS files

In [2]:
# Class definition for FundamentusBot
class FundamentusBot:
    # Constructor to configure the browser
    def __init__(self):
        # Configure Chrome options with download preferences (always ask where to save the file)
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_experimental_option("prefs", {
            "download.prompt_for_download": True,
            "safebrowsing.enabled": True
        })
        #create the service
        service = Service(ChromeDriverManager().install())
        #create and open the browser
        self.browser = webdriver.Chrome(service=service, options=chrome_options)

    # Method to open the Fundamentus website and navigate to the desired data section
    def open_site(self):
        #navigate to initial page of Fundamentus
        self.browser.get("https://fundamentus.com.br/")
        time.sleep(2) #wait 2 seconds 
        #click to open the Fundamentus Mobile
        self.browser.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[2]/ul[1]/li[5]/a').click()
        time.sleep(3) #wait
        #call the method open_data
        self.open_data()

    # Method to navigate to the data section and process data for a list of companies
    def open_data(self):
        #list of companies of interest
        companies = config.companies
        #iterate the companies list
        for company in companies:
            #get company
            self.company = company
            #call the method that download the information
            self.dwnld_company_info()
        time.sleep(5) #wait
        # call the method that move the .zip files from 'source_path_download' to 'destination_path_zip'
        self.move_files()
        # call the method that extract the XLS files from the .zip and save in 'destination_path_xls'
        self.extract_files()
        # close the browser
        self.browser.quit()

    # Method to download company information from the Fundamentus website
    def dwnld_company_info(self):
        #try to do the sequence of actions
        try:
            # find the search bar and type the company
            self.browser.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[2]/form/input[1]').send_keys(self.company)
            time.sleep(2) #wait
            # press enter to open the company's data
            self.browser.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[2]/form/input[1]').send_keys(Keys.ENTER)
            time.sleep(2) #wait
            # click in 'Informações'
            self.browser.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[2]/ul/li[3]/span').click()
            time.sleep(2) #wait
            # click in "Balanços em EXCEL"
            self.browser.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div/div[2]/ul/li[3]/ul/li[10]').click()
            time.sleep(3) #wait
            #get the code of the company 
            paper =self.browser.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div[1]/h1').text
            time.sleep(3) #wait
            # click in "baixar" to download the file
            self.browser.find_element(By.XPATH, '//*[@id="form-planilha"]/a').click()
            time.sleep(3) #wait
            # typing a new name for the downloadable file
            pyautogui.typewrite(f'bal_{paper}.zip')
            time.sleep(3) #wait
            # press ENTER to complete the download
            pyautogui.press('enter')
        # in case of any problem during the actions (try), the exception runs
        except Exception as e:
            # show the errors
            print(f'Error: {e}')
            # print a message
            print(f'There was an errror with the company: {self.company}')

    # Method to move downloaded files to a specified destination
    def move_files(self):
        #get all the files from "download" folder
        files_list = [f for f in os.listdir(config.source_path_download) if os.path.isfile(os.path.join(config.source_path_download, f))]
        # iterate all the files
        for file in files_list:
            # get the first 3 characters of the file name
            file_name = file[0:3]
            # check if the file name starts with 'bal' 
            if file_name == 'bal':
                # Check if the destination file exists
                if os.path.exists(f'{config.destination_path_zip}/{file}'):
                    # If it exists, remove it
                    os.remove(f'{config.destination_path_zip}/{file}')
                # move the file to the desired destination
                shutil.move(f'{config.source_path_download}/{file}', config.destination_path_zip)


    # Method to extract files from ZIP archives
    def extract_files(self):
        # get all the files from the directory
        files_list = [f for f in os.listdir(config.destination_path_zip) if os.path.isfile(os.path.join(config.destination_path_zip, f))]
        # iterate each file
        for file in files_list:
            # get the first 3 characters of the file name
            file_name = file[0:3]
            # check if the file name starts with 'bal' 
            if file_name == 'bal': 
                # Check if the destination file exists
                if os.path.exists(f'{config.destination_path_xls}/{file[0:-4]}.xls'):
                    # If it exists, remove it
                    os.remove(f'{config.destination_path_xls}/{file[0:-4]}.xls') 
                # Using the 'with' statement to open the ZIP archive for reading ('r' mode)
                # The archive file path is constructed using the destination path for ZIP files and the current file in the loop
                with zipfile.ZipFile(f'{config.destination_path_zip}/{file}', 'r') as zip_ref:
                    # Extract the specific file to the specified folder
                    zip_ref.extract('balanco.xls',f'{config.destination_path_xls}/')
                    # Rename the file
                    source_path = f'{config.destination_path_xls}/' #path of the file
                    file_to_rename = 'balanco.xls' #current file name
                    new_filename = f'{file[0:-4]}.xls' #desired file name
                    old_file_path = os.path.join(source_path,file_to_rename) #old file
                    new_file_path = os.path.join(source_path, new_filename) #new file
                    os.rename(old_file_path, new_file_path) #rename
                
# Create an instance of FundamentusBot and execute the process
bot = FundamentusBot()
bot.open_site()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.72)
Stacktrace:
	GetHandleVerifier [0x01006EE3+174339]
	(No symbol) [0x00F30A51]
	(No symbol) [0x00C46FF6]
	(No symbol) [0x00C2EFE7]
	(No symbol) [0x00C9B53B]
	(No symbol) [0x00CA9E7B]
	(No symbol) [0x00C96DA6]
	(No symbol) [0x00C71034]
	(No symbol) [0x00C71F8D]
	GetHandleVerifier [0x010A4B1C+820540]
	sqlite3_dbdata_init [0x011653EE+653550]
	sqlite3_dbdata_init [0x01164E09+652041]
	sqlite3_dbdata_init [0x011597CC+605388]
	sqlite3_dbdata_init [0x01165D9B+656027]
	(No symbol) [0x00F3FE6C]
	(No symbol) [0x00F383B8]
	(No symbol) [0x00F384DD]
	(No symbol) [0x00F25818]
	BaseThreadInitThunk [0x750DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76FE7A4E+286]
	RtlGetAppContainerNamedObjectPath [0x76FE7A1E+238]


## Step 4 (Data Processing)

### Getting the data from XLS files to process with pandas

For this first moment, let's just get 1 sample of balance and income (using "break" in the end of the code) to understand the changes that need to be done before inserting the data into a dictionary

In [3]:
#get all the files available (xls files)
files = os.listdir(config.destination_path_xls)
# iterate for ech file
for file in files:
    #create the file_name to be the key of the dictionary
    if '11' in file: #some codes has 11
        file_name = file[-10:-4] #get the code
    else:
        file_name = file[-9:-4] #get the code
    #check if the file_name is in the list  companies (this is to help in case we want to analyze only some campanies)
    if file_name in config.companies:
        #get the company balance
        balance = pd.read_excel(f'./balances/extracted/{file}', sheet_name=0)
        #get the company income statement
        income = pd.read_excel(f'./balances/extracted/{file}', sheet_name=1)
        break #to get only one result


WARNING *** file size (67047) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (67047) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


### Let's see the head of both tables to understand what need to be done first

In [4]:
balance.head()

,"XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data",Balanço Patrimonial - AMBEV S/A,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,NaN,31/12/2012,31/03/2013,30/06/2013,30/09/2013,31/12/2013,31/03/2014,30/06/2014,30/09/2014,31/12/2014,...,30/06/2021,30/09/2021,31/12/2021,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023
1,Ativo Total,1346301.056,1340374.016,58739269.632,59618975.744,68674015.232,63297044.48,62989045.76,65125920.768,72143200.256,...,124440133.632,135133249.536,138602479.616,127399919.616,136633409.536,142063960.064,137958080.512,135466721.28,133294415.872,137914204.16
2,Ativo Circulante,71641,77552,12057052.16,12478373.888,20470011.904,16352306.176,15773268.992,15446576.128,20728420.352,...,32705665.024,38197080.064,38627139.584,34479796.224,38238560.256,41556963.328,37816713.216,35378688,34324092.928,37552668.672
3,Caixa e Equivalentes de Caixa,48155,74204,4482174.976,4835169.792,11285832.704,7296176.128,6273862.144,5748115.968,9722066.944,...,13269346.304,17956171.776,16627697.664,12887921.664,14129258.496,17712654.336,14926435.328,12214085.632,12117013.504,17413906.432
4,Aplicações Financeiras,0,0,486132.992,612489.024,288604,410172.992,379937.984,526788.992,712958.016,...,1245607.04,2044573.952,1914606.976,1345730.048,1535714.048,1347216,454496.992,365284,313504,227164


In [5]:
income.head(5)

,"XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data",Demonstrativo de Resultado - AMBEV S/A - Trimestres Isolados,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,,NaN,31/03/2013,30/06/2013,30/09/2013,31/12/2013,31/03/2014,30/06/2014,30/09/2014,31/12/2014,...,30/06/2021,30/09/2021,31/12/2021,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023
1,Receita Bruta de Vendas e/ou Serviços,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Deduções da Receita Bruta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Receita Líquida de Vendas e/ou Serviços,NaN,10617,7503133.184,8462602.752,18815037.44,9045071.872,8177433.088,8624396.288,12232882.176,...,15711140.864,18492608.512,22010836.992,18439151.616,17988995.072,20587642.88,22693033.984,20531744.768,18898114.56,20317763.584
4,Custo de Bens e/ou Serviços Vendidos,NaN,0,-2626988.032,-2834846.976,-5935966.208,-3008314.112,-3040666.112,-2955760.128,-3809847.552,...,-7965268.992,-9253070.848,-10496073.728,-9414486.016,-9374254.08,-10648073.216,-10985254.912,-10131684.352,-9635608.576,-10223017.984


### It is possible to see that in both dataframes we need to:

- Insert the company code in the first column (header)
- Transform the first row in a the header
- Transform first column into index

In [6]:
# Insert the company code in the first column (header)
balance.iloc[0,0] = file_name
income.iloc[0,0] = file_name

In [7]:
# Transform the first row in a the header
balance.columns = balance.iloc[0] #insert first row as header
balance = balance[1:] #get first row until the end (drop the line 0 that was duplicated)
income.columns = income.iloc[0] #insert first row as header
income = income[1:] #get first row until the end (drop the line 0 that was duplicated)

In [8]:
# Transform first column into index
balance = balance.set_index(file_name)
income = income.set_index(file_name)

### Let's take a look in the columns

In [9]:
#show columns
print(balance.columns)
print(income.columns)

Index(['31/12/2012', '31/03/2013', '30/06/2013', '30/09/2013', '31/12/2013',
       '31/03/2014', '30/06/2014', '30/09/2014', '31/12/2014', '31/03/2015',
       '30/06/2015', '30/09/2015', '31/12/2015', '31/03/2016', '30/06/2016',
       '30/09/2016', '31/12/2016', '31/03/2017', '30/06/2017', '30/09/2017',
       '31/12/2017', '31/03/2018', '30/06/2018', '30/09/2018', '31/12/2018',
       '31/03/2019', '30/06/2019', '30/09/2019', '31/12/2019', '31/03/2020',
       '30/06/2020', '30/09/2020', '31/12/2020', '31/03/2021', '30/06/2021',
       '30/09/2021', '31/12/2021', '31/03/2022', '30/06/2022', '30/09/2022',
       '31/12/2022', '31/03/2023', '30/06/2023', '30/09/2023'],
      dtype='object', name=0)
Index([         nan, '31/03/2013', '30/06/2013', '30/09/2013', '31/12/2013',
       '31/03/2014', '30/06/2014', '30/09/2014', '31/12/2014', '31/03/2015',
       '30/06/2015', '30/09/2015', '31/12/2015', '31/03/2016', '30/06/2016',
       '30/09/2016', '31/12/2016', '31/03/2017', '30/06/201

It is possible to see that the balance contains data from 31/12/2012 and income dataframe starts in 31/03/2013

So let's drop the first column of balance table, this way both will have data from the same period. For the complete data we will do it later, to have the big picture of the time ranges. 

In [10]:
#drop first columns of balance
balance = balance.drop(columns=balance.columns[0], axis=1)
#drop first column of income (used a different methos due the nan name of the column)
del income[income.columns[0]]

In [11]:
#show columns
print(balance.columns)
print(income.columns)

Index(['31/03/2013', '30/06/2013', '30/09/2013', '31/12/2013', '31/03/2014',
       '30/06/2014', '30/09/2014', '31/12/2014', '31/03/2015', '30/06/2015',
       '30/09/2015', '31/12/2015', '31/03/2016', '30/06/2016', '30/09/2016',
       '31/12/2016', '31/03/2017', '30/06/2017', '30/09/2017', '31/12/2017',
       '31/03/2018', '30/06/2018', '30/09/2018', '31/12/2018', '31/03/2019',
       '30/06/2019', '30/09/2019', '31/12/2019', '31/03/2020', '30/06/2020',
       '30/09/2020', '31/12/2020', '31/03/2021', '30/06/2021', '30/09/2021',
       '31/12/2021', '31/03/2022', '30/06/2022', '30/09/2022', '31/12/2022',
       '31/03/2023', '30/06/2023', '30/09/2023'],
      dtype='object', name=0)
Index(['31/03/2013', '30/06/2013', '30/09/2013', '31/12/2013', '31/03/2014',
       '30/06/2014', '30/09/2014', '31/12/2014', '31/03/2015', '30/06/2015',
       '30/09/2015', '31/12/2015', '31/03/2016', '30/06/2016', '30/09/2016',
       '31/12/2016', '31/03/2017', '30/06/2017', '30/09/2017', '31/12/201

### Now we are ready to collect the data of all the companies and insert into the dictionary

In [260]:
#create a dictionary to store the data
fundamentus = {}
#get all the files available (xls files)
files = os.listdir(config.destination_path_xls)
# iterate for ech file
for file in files:
    #create the file_name to be the key of the dictionary
    if '11' in file: #some codes has 11
        file_name = file[-10:-4] #get the code
    else:
        file_name = file[-9:-4] #get the code
    #check if the file_name is in the list  companies (this is to help in case we want to analyze only some campanies)
    if file_name in config.companies:
        #get the company balance
        balance = pd.read_excel(f'./balances/extracted/{file}', sheet_name=0)
        #get the company income statement
        income = pd.read_excel(f'./balances/extracted/{file}', sheet_name=1)
        # Insert the company code in the first column (header)
        balance.iloc[0,0] = file_name
        income.iloc[0,0] = file_name
        # Transform the first row in a the header
        balance.columns = balance.iloc[0] #insert first row as header
        balance = balance[1:] #get first row until the end (drop the line 0 that was duplicated)
        income.columns = income.iloc[0] #insert first row as header
        income = income[1:] #get first row until the end (drop the line 0 that was duplicated)
        # Transform first column into index
        balance = balance.set_index(file_name)
        income = income.set_index(file_name)
        # Insert data into the dictionary
        #fundamentus[file_name] = balance._append(income)  
        fundamentus[file_name] = pd.concat([balance,income])
        # drop the columns name 'nan'
        for i in range(0,len(fundamentus[file_name].columns)): #repeat from 0 to len(columns)
            # need the try due and error on the check for non numeric columns headers
            try:
                if math.isnan(fundamentus[file_name].columns[i]): #check if it is a nan
                    # drop columns in case of yes
                    fundamentus[file_name] = fundamentus[file_name].drop(columns=fundamentus[file_name].columns[i])
            # do nothing in case of error on the try
            except: 
                None # do nothing

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (45071) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (45071) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (82419) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (82419) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (90643) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is no

### Getting the historical prices of shares

Selecting the end date (date of the last balance of companies) and creating the start date (getting last 5 years of data)

In [138]:
# end date 
from datetime import datetime, timedelta #library needed to deal with times
end_date = fundamentus[config.companies[1]].columns[-1] # get the last column (last date)
end_date_dt = datetime.strptime(end_date, "%d/%m/%Y") #trasnform string to datetime
start_date_dt = (end_date_dt - timedelta(days= 5 * 365)) #Create the start_date based on time variation 
end_date_str = end_date_dt.strftime("%Y-%m-%d") #transform end_date to string to match the needed format
start_date_str = start_date_dt.strftime("%Y-%m-%d") # transform start_date to string to match the needed format
print(f'Start date: {end_date_str}') #print
print(f'End date: {start_date_str}') #print

Start date: 2023-09-30
End date: 2018-10-01


Get the cotations for the desired companies

In [139]:
cotation = {} #create a dictionary
# get the cotation of the desired companies
for company in  config.companies: # loop to get the cotations
    cotation_df = yf.download(f"{company}.SA", start=start_date_str, end=end_date_str, progress=False) # get the cotation
    cotation_df['Company'] = company # insert the company name
    cotation[company] = cotation_df #insert dataframe to the dictionary

cotation_bkp = cotation # create a dataframe for backup (only during development)


1 Failed download:
['BRML3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['HGTX3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['CPLE6.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-10-01 -> 2023-09-30)')

1 Failed download:
['CSAN3.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-10-01 -> 2023-09-30)')

1 Failed download:
['CPFE3.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-10-01 -> 2023-09-30)')

1 Failed download:
['CVCB3.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-10-01 -> 2023-09-30)')

1 Failed download:
['CYRE3.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-10-01 -> 2023-09-30)')

1 Failed download:
['ECOR3.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-10-01 -> 2023-09-30)')

1 Failed downlo

Compare and correct the period of the fundamentus data and cotations (the period need to match - so we want analyze last 5 years)

In [140]:
list(fundamentus.keys())[0]

'ABEV3'

In [141]:
# look to the time period of fundamentus data (based on the first company in the dictionary (AMBEV - ABEV3))
fundamentus[list(fundamentus.keys())[0]].columns

Index(['31/12/2012', '31/03/2013', '30/06/2013', '30/09/2013', '31/12/2013',
       '31/03/2014', '30/06/2014', '30/09/2014', '31/12/2014', '31/03/2015',
       '30/06/2015', '30/09/2015', '31/12/2015', '31/03/2016', '30/06/2016',
       '30/09/2016', '31/12/2016', '31/03/2017', '30/06/2017', '30/09/2017',
       '31/12/2017', '31/03/2018', '30/06/2018', '30/09/2018', '31/12/2018',
       '31/03/2019', '30/06/2019', '30/09/2019', '31/12/2019', '31/03/2020',
       '30/06/2020', '30/09/2020', '31/12/2020', '31/03/2021', '30/06/2021',
       '30/09/2021', '31/12/2021', '31/03/2022', '30/06/2022', '30/09/2022',
       '31/12/2022', '31/03/2023', '30/06/2023', '30/09/2023'],
      dtype='object', name=0)

In [142]:
# Converting the index as date
#fundamentus['ABEV3'].index = pd.to_datetime(fundamentus['ABEV3'].index)
type(cotation[list(fundamentus.keys())[0]].index.values[1])
date = cotation[list(fundamentus.keys())[0]].index.values[1]
print(date)
# Converter para objeto datetime
data_datetime = np.datetime_as_string(date, unit='D')  # 'D' representa dias
print(data_datetime)
# Formatar a data no formato desejado
data_formatada = datetime.strptime(data_datetime, "%Y-%m-%d").strftime("%d/%m/%Y")
print(data_formatada)

2018-10-02T00:00:00.000000000
2018-10-02
02/10/2018


Filling the needed date range

In [143]:
# organize the columns of fundamentus to same format as cotation date
cols = fundamentus[list(fundamentus.keys())[0]].columns #get the columns of first dataframe (key = ABEV3)
new_cols = [] #create a list to store the new columns names
for col in cols: #loop
    # convert string to datetime object
    col = datetime.strptime(col, "%d/%m/%Y")
    # convert datetime object to numpy.datetime64 (as in the cotations dataframe)
    col = np.datetime64(col)
    #save in the new_cols list
    new_cols.append(col)

# transform start date to numpy.datetime64 format to compare all the dates
start_date_np = datetime.strptime(start_date_dt.strftime("%d/%m/%Y"), "%d/%m/%Y") #correct the format
start_date_np = np.datetime64(start_date_np) #transform to numpy datetime 64 bits
new_cols_remove = [] #create as new list to insert the cols that need to be deleted from fundamentus dataframe
for col in new_cols: #loop
    if col < start_date_np: # check if the col in fundamentus table is lower (older) than start date
        new_cols_remove.append(col) # in case of yes, select this col to be deleted from fundamentus

new_cols = [item for item in new_cols if item not in new_cols_remove] #update new columns for fundamentus

#show neww columns
display(new_cols)
# convert string to datetime object
new_cols_dt = np.datetime_as_string(date, unit='D')  # 'D' represent days
print(new_cols_dt) #print
# Select the desired date format
new_cols_dt = datetime.strptime(new_cols_dt, "%Y-%m-%d").strftime("%d/%m/%Y")
print(new_cols_dt) #print

#index
x = 0
#new_cols = new_cols
# loop the check all the index from cotation
for col in new_cols:
    # loop for each key in the cotation dictionary
    for company in list(fundamentus.keys()):
        # if the date from fundamentus is in cotation, do nothing
        if col in cotation[company].index:
            print('OK')
        # if the date is not in the cotation, need to create
        else: 
            # print message
            print(f'Need to add {col} in the cotation table')
            # increment index
            x += 1
            # loop to subtract days until find a valid one to get the cotation
            for sub_days in range (1,10):
                # subtract 1 day
                days = np.timedelta64(sub_days, 'D') #create the variabele with 1 day
                date_sub_x_days = col - days # subtract 1 day
                # check if the new data exists in cotation to get the price
                if date_sub_x_days in cotation[company].index: 
                    #once the date is found, get the price and create the needed date
                    new_line_data = pd.Series(name=col, data = cotation[company].loc[date_sub_x_days])
                    print(new_line_data)
                    break
            # add the new line with the needed date to the cotation dataframe
            cotation[company] = cotation[company]._append(new_line_data)
            print('Added') #print

[numpy.datetime64('2018-12-31T00:00:00.000000'),
 numpy.datetime64('2019-03-31T00:00:00.000000'),
 numpy.datetime64('2019-06-30T00:00:00.000000'),
 numpy.datetime64('2019-09-30T00:00:00.000000'),
 numpy.datetime64('2019-12-31T00:00:00.000000'),
 numpy.datetime64('2020-03-31T00:00:00.000000'),
 numpy.datetime64('2020-06-30T00:00:00.000000'),
 numpy.datetime64('2020-09-30T00:00:00.000000'),
 numpy.datetime64('2020-12-31T00:00:00.000000'),
 numpy.datetime64('2021-03-31T00:00:00.000000'),
 numpy.datetime64('2021-06-30T00:00:00.000000'),
 numpy.datetime64('2021-09-30T00:00:00.000000'),
 numpy.datetime64('2021-12-31T00:00:00.000000'),
 numpy.datetime64('2022-03-31T00:00:00.000000'),
 numpy.datetime64('2022-06-30T00:00:00.000000'),
 numpy.datetime64('2022-09-30T00:00:00.000000'),
 numpy.datetime64('2022-12-31T00:00:00.000000'),
 numpy.datetime64('2023-03-31T00:00:00.000000'),
 numpy.datetime64('2023-06-30T00:00:00.000000'),
 numpy.datetime64('2023-09-30T00:00:00.000000')]

2018-10-02
02/10/2018
Need to add 2018-12-31T00:00:00.000000 in the cotation table
Open             15.23
High             15.67
Low               15.2
Close            15.38
Adj Close    12.625372
Volume        15498800
Company          ABEV3
Name: 2018-12-31T00:00:00.000000, dtype: object
Added
Need to add 2018-12-31T00:00:00.000000 in the cotation table
Open              35.0
High              36.0
Low          34.619999
Close             36.0
Adj Close         36.0
Volume         2607400
Company          AZUL4
Name: 2018-12-31T00:00:00.000000, dtype: object
Added
Need to add 2018-12-31T00:00:00.000000 in the cotation table
Open         8.766666
High         9.033333
Low          8.763333
Close        8.936666
Adj Close    7.502092
Volume       36834600
Company         B3SA3
Name: 2018-12-31T00:00:00.000000, dtype: object
Added
Need to add 2018-12-31T00:00:00.000000 in the cotation table
Open         45.189999
High         46.490002
Low          44.849998
Close        46.490002
Adj 

In [144]:
cotation.keys()

dict_keys(['ABEV3', 'AZUL4', 'B3SA3', 'BBSE3', 'BRML3', 'BBDC4', 'BRAP4', 'BBAS3', 'BRKM5', 'BRFS3', 'BPAC11', 'CRFB3', 'CCRO3', 'CMIG4', 'HGTX3', 'CIEL3', 'COGN3', 'CPLE6', 'CSAN3', 'CPFE3', 'CVCB3', 'CYRE3', 'ECOR3', 'ELET6', 'EMBR3', 'ENBR3', 'ENGI11', 'ENEV3', 'EGIE3', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'NTCO3', 'HAPV3', 'HYPE3', 'IGTA3', 'GNDI3', 'ITSA4', 'ITUB4', 'JBSS3', 'JHSF3', 'KLBN11', 'RENT3', 'LCAM3', 'LAME4', 'LREN3', 'MGLU3', 'MRFG3', 'BEEF3', 'MRVE3', 'MULT3', 'PCAR3', 'PETR4', 'BRDT3', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'SBSP3', 'SANB11', 'CSNA3', 'SULA11', 'SUZB3', 'TAEE11', 'VIVT3', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5', 'VALE3', 'VVAR3', 'WEGE3', 'YDUQ3'])

Compare and check if we have cotation for all the companies in fundamentus e drop those that are not in both datasets

In [145]:
#check difference 
print(f'Quantity companyes in fundamentus: {len(list(fundamentus.keys()))}')
print(f'Quantity companyes in cotation: {len(list(cotation.keys()))}')

Quantity companyes in fundamentus: 74
Quantity companyes in cotation: 76


In [146]:
# Companies in fundamentus but not in cotation
companies_to_drop= [i for i in list(fundamentus.keys()) if (i not in list(cotation.keys()))]
#drop companies in fundamentus
for company in companies_to_drop:
    fundamentus.pop(company)

# Companies in cotation but not in fundamentus
companies_to_drop= [i for i in list(cotation.keys()) if (i not in list(fundamentus.keys()))]
#drop companies in fundamentus
for company in companies_to_drop:
    cotation.pop(company)

In [147]:
#check difference 
print(f'Quantity companyes in fundamentus: {len(list(fundamentus.keys()))}')
print(f'Quantity companyes in cotation: {len(list(cotation.keys()))}')

Quantity companyes in fundamentus: 74
Quantity companyes in cotation: 74


 Drop companies with null values in cotation

In [148]:
#list of companies
list_companies = list(cotation.keys())
# loop to check all companies
for company in list_companies:
    # check if there are nulls
    if cotation[company].isnull().values.any():
        #drop company
        cotation.pop(company)
        fundamentus.pop(company)

In [162]:
#check difference 
print(f'Quantity companyes in fundamentus: {len(list(fundamentus.keys()))}')
print(f'Quantity companyes in cotation: {len(list(cotation.keys()))}')

Quantity companyes in fundamentus: 74
Quantity companyes in cotation: 74


In [ ]:
fundamentus['B3SA3']

Merge fundamentus and cotation into the same dictionary

In [189]:
# create dictionary to store all the data
historic = {}
# Loop to sweep all the companies
for company in list_companies:
    # creating a table
    table_fundamentus = fundamentus[company].T #insert dates in index
    # changing index to datetime python
    table_fundamentus.index = pd.to_datetime(table_fundamentus.index, format="%d/%m/%Y")
    # get cotation - only adj close is desired
    table_cotation = cotation[company]
    table_cotation = table_cotation[['Adj Close']]
    # final table
    table = table_fundamentus.merge(table_cotation, right_index=True, left_index=True) #only lines in both tables at same time
    # insert the company in the index header
    table.index_name = company
    # save ina new dataframe
    historic[company] = table

In [191]:
fundamentus['B3SA3']

,31/12/2008,31/03/2009,30/06/2009,30/09/2009,31/12/2009,31/03/2010,30/06/2010,30/09/2010,31/12/2010,31/03/2011,...,30/06/2021,30/09/2021,31/12/2021,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023
B3SA3,,,,,,,,,,,,,,,,,,,,,
20753426.432,20430088.192,21204279.296,20936243.2,21292857.344,21201184.768,21615536.128,21304356.864,22771748.864,22633975.808,24032278.528,...,49530806.272,54807453.696,52531769.344,50737246.208,47394320.384,48771391.488,47594770.432,45806399.488,47176957.952,47872684.032
2408035.072,1965460.992,2814573.056,2569664,2936913.92,2778968.064,4192899.072,3786267.904,3131595.008,2547589.12,3779795.968,...,19873683.456,25600839.68,21095032.832,19784017.92,16721852.416,18498359.296,17129612.288,15579206.656,16141927.424,17212887.04
2055229.056,1784295.936,2592073.984,2335424,2701556.992,2650563.072,4038099.968,3619712,2960681.984,104017,1542301.056,...,2065184,3777044.992,2560516.096,2711499.008,2371543.04,4267671.04,2613794.048,1929953.024,2170661.888,3174522.88
0,0,0,0,0,0,0,0,0,2264408.064,2031428.992,...,16274057.216,20196892.672,16576854.016,15857626.112,12803062.784,12481081.344,12290716.672,12239222.784,12092050.432,11756456.96
329968.992,163303.008,196336.992,206844,209856,97036,129052,140072,142880.992,51399,67326,...,391166.016,369465.984,436257.984,417460.992,431342.016,452375.008,503840,464980,504895.008,475030.016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IR Diferido,32961,-7237,-152196,-60196,-116961.016,-108310,-101836,-125761,-106713.992,-111167,...,-323121.984,-146460,-92729.056,-379316,-30583,77418,36949.992,-41253,-27164,-60232
Participações/Contribuições Estatutárias,0,0,0,0,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Reversão dos Juros sobre Capital Próprio,0,0,0,0,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Remove tables that have different (non-standard) columns

In [261]:
#list of companies
list_companies = list(cotation.keys())
# create columns list
cols = list(historic[list(fundamentus.keys())[0]].columns) # get cols of first company as standard
print(len(list_companies))
# look all the companies to check the columns
for company in list_companies:
    # compare the sets
    if set(cols) != set(historic[company].columns):
        print(company)
        print("Different")
        print(cols)
        print(historic[company].columns)
        # drop if the sets are different
        historic.pop(company)

#update the companies list
list_companies = list(historic.keys())


74
B3SA3
Different
['Ativo Total', 'Ativo Circulante', 'Caixa e Equivalentes de Caixa', 'Aplicações Financeiras', 'Contas a Receber', 'Estoques', 'Ativos Biológicos', 'Tributos a Recuperar', 'Despesas Antecipadas', 'Outros Ativos Circulantes', 'Ativo Realizável a Longo Prazo', 'Aplicações Financeiras Avaliadas a Valor Justo', 'Aplicações Financeiras Avaliadas ao Custo Amortizado', 'Contas a Receber', 'Estoques', 'Ativos Biológicos', 'Tributos Diferidos', 'Despesas Antecipadas', 'Créditos com Partes Relacionadas', 'Outros Ativos Não Circulantes', 'Investimentos', 'Imobilizado', 'Intangível', 'Diferido', 'Passivo Total', 'Passivo Circulante', 'Obrigações Sociais e Trabalhistas', 'Fornecedores', 'Obrigações Fiscais', 'Empréstimos e Financiamentos', 'Passivos com Partes Relacionadas', 'Dividendos e JCP a Pagar', 'Outros', 'Provisões', 'Passivos sobre Ativos Não-Correntes a Venda e Descontinuados', 'Passivo Não Circulante', 'Empréstimos e Financiamentos', 'Passivos com Partes Relacionadas',

Selecting the end date (date of the last balance of companies) and creating the start date (getting last 5 years of data)